In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
osm_nodes_table_name="osm_nodes"
osm_intersections_table_name="osm_intersections"
2

2

In [17]:
DELETE_TABLES=True
if DELETE_TABLES:
    try:
        cmd="""DROP TABLE `%s`"""%osm_intersections_table_name
        cursor.execute(cmd)
    except Exception as e:
        print(e)
        pass
    cmd="""DROP TABLE `%s`"""%osm_nodes_table_name
    cursor.execute(cmd)
2

1051 (42S02): Unknown table 'dem_from_contours_lines.osm_intersections'


ProgrammingError: 1051 (42S02): Unknown table 'dem_from_contours_lines.osm_nodes'

In [18]:
CREATE_TABLES=True
if CREATE_TABLES:
    cmd="""CREATE TABLE `%s`(
    `osm_id` BIGINT NOT NULL,
    `contour_line_id` INT NOT NULL,
    `geometry` POINT NOT NULL,
    `elevation` FLOAT,

    FOREIGN KEY (contour_line_id) REFERENCES %s(id),
    PRIMARY KEY (osm_id)
    );
        """%(osm_nodes_table_name,contours_lines_table_name)
    
    cursor.execute(cmd)



    
    cmd="""CREATE TABLE `%s`(
        `osm_begin` BIGINT NOT NULL,
        `osm_end` BIGINT NOT NULL,
        `osm_key` INT NOT NULL,
        `contour_line_id` INT NOT NULL,
        `edge_coordinate` FLOAT NOT NULL,
        `geometry` POINT NOT NULL,
        `elevation` FLOAT NOT NULL,

        FOREIGN KEY (osm_begin) REFERENCES %s(osm_id),
        FOREIGN KEY (osm_end) REFERENCES %s(osm_id),
        FOREIGN KEY (contour_line_id) REFERENCES %s(id),
        PRIMARY KEY (osm_begin,osm_end,osm_key,edge_coordinate)
        

          )
          ENGINE=InnoDB;
        """%(osm_intersections_table_name,osm_nodes_table_name,osm_nodes_table_name,contours_lines_table_name)
    cursor.execute(cmd)
    

2

2

In [5]:
t1=time.time()
G=build_graph(cursor)
max_depth=np.max([data['depth'] for _,data in G.nodes(data=True)])
t2=time.time()
print(t2-t1)

0.7671797275543213


In [6]:
t1=time.time()
place_name='Corse, France'
G_osm=ox.graph_from_place(place_name,network_type='drive')
G_osm=to_multi_graph(G_osm)
osm_crs=ox.graph_to_gdfs(G_osm,edges=False).estimate_utm_crs()
G_osm=ox.project_graph(G_osm,to_crs=osm_crs)
add_missing_geometries(G_osm)
t2=time.time()
print(t2-t1)
len(G_osm.nodes()),len(G_osm.edges())

72.91492652893066


(17435, 20792)

In [7]:
t1=time.time()
osm_edges_df=ox.graph_to_gdfs(G_osm,nodes=False,edges=True)
osm_edges_df['edge']=osm_edges_df.index
osm_edges_df=osm_edges_df.loc[:,['edge','geometry']]
osm_edges_df['number_of_points']=osm_edges_df['geometry'].apply(lambda ls:len(ls.coords))
#osm_edges_df=osm_edges_df.sort_values(by='length')
osm_edges_df=osm_edges_df.set_index(np.arange(len(osm_edges_df)))
osm_edges_df['id_edge']=osm_edges_df.index
t2=time.time()
print(t2-t1)
osm_edges_df

0.3860785961151123


,edge,geometry,number_of_points,id_edge
0,"(14468319, 15905397, 0)","LINESTRING (525532.710 4599879.701, 525543.715...",82,0
1,"(14468319, 1434053206, 0)","LINESTRING (525532.710 4599879.701, 525519.651...",4,1
2,"(14468319, 1434053161, 0)","LINESTRING (525532.710 4599879.701, 525532.752...",20,2
3,"(15905397, 1976904946, 0)","LINESTRING (526252.098 4600660.017, 526286.419...",12,3
4,"(15905397, 1976906939, 0)","LINESTRING (526252.098 4600660.017, 526253.327...",7,4
...,...,...,...,...
20787,"(9744995304, 9744995311, 0)","LINESTRING (496541.756 4720170.475, 496521.473...",2,20787
20788,"(9744995304, 9744995310, 0)","LINESTRING (496541.756 4720170.475, 496558.928...",7,20788
20789,"(9964974606, 9964974609, 0)","LINESTRING (494174.708 4719764.250, 494163.668...",4,20789
20790,"(9964974610, 9964974611, 0)","LINESTRING (494100.379 4719512.527, 494038.489...",2,20790


In [8]:
t1=time.time()
intersection,depth=decreasing_depth_intersections(osm_edges_df,osm_crs,G,cursor,max_depth,depth_step=20,max_delta_time_total=float('inf'),max_delta_time_per_level=float('inf'))
t2=time.time()
print('total : %f'%(t2-t1),depth)

520<depth<=540 :0.101366 
500<depth<=520 :0.112132 
480<depth<=500 :0.243263 
460<depth<=480 :0.295815 
440<depth<=460 :0.441626 
420<depth<=440 :0.741260 
400<depth<=420 :1.083950 
380<depth<=400 :1.634173 
360<depth<=380 :2.029849 
340<depth<=360 :2.437088 
320<depth<=340 :2.920865 
300<depth<=320 :3.445841 
280<depth<=300 :4.008569 
260<depth<=280 :4.665973 
240<depth<=260 :5.561779 
220<depth<=240 :6.611834 
200<depth<=220 :8.109791 
180<depth<=200 :9.607444 
160<depth<=180 :11.860657 
140<depth<=160 :14.788711 
120<depth<=140 :16.806210 
100<depth<=120 :21.576238 
80<depth<=100 :28.066434 
60<depth<=80 :31.102396 
40<depth<=60 :32.993271 
20<depth<=40 :36.867280 
0<depth<=20 :56.114423 
total : 304.233359 1


In [9]:
t1=time.time()
pre_osm_nodes_to_contours={}
for edge,df in intersection.groupby('edge'):
    df=df.sort_values('edge_coordinate')
    u,v=edge[:2]
    if not u in pre_osm_nodes_to_contours:
        pre_osm_nodes_to_contours[u]={df.iloc[0]['id']}
    else:
        pre_osm_nodes_to_contours[u].add(df.iloc[0]['id'])
    if not v in pre_osm_nodes_to_contours:
        pre_osm_nodes_to_contours[v]={df.iloc[-1]['id']}
    else:
        pre_osm_nodes_to_contours[v].add(df.iloc[-1]['id'])
t2=time.time()
print(t2-t1)

20.456740140914917


In [10]:
t1=time.time()
G_osm.remove_edges_from(intersection['edge'].unique())
t2=time.time()
print(t2-t1)

0.040374040603637695


In [11]:
t1=time.time()
CC=list(nx.connected_components(nx.Graph(G_osm)))
pre_CC_to_contours={}
for k,cc in enumerate(CC):
    contours_ids=set()
    for osm_node in cc:
        if osm_node in pre_osm_nodes_to_contours:
            contours_ids=contours_ids.union(pre_osm_nodes_to_contours[osm_node])
    pre_CC_to_contours[k]=contours_ids
    if len(contours_ids)==0:
        print('wesh')
t2=time.time()
print(t2-t1)

0.2853882312774658


In [12]:
t1=time.time()
CC_to_contours={}
ambiguous_contours={}
for k,contour_ids in pre_CC_to_contours.items():
    if len(contour_ids)>1:
        id_1,id_2=list(contour_ids)[:2]
        if id_2 in G.neighbors(id_1):
            CC_to_contours[k]=id_1
        else:
            predecessors=list(G.predecessors(id_2))
            contour_id=id_2 if len(predecessors)==0 else predecessors[0]
            CC_to_contours[k]=contour_id
    else:
        contour_id=list(contour_ids)[0]
        if not(contour_id) in ambiguous_contours:
            ambiguous_contours[contour_id]=[k]
        else:
            ambiguous_contours[contour_id].append(k)
t2=time.time()
print(t2-t1)

0.03941059112548828


In [13]:
contour_id_bunch_size=10
t1=time.time()

contour_ids=list(ambiguous_contours.keys())
for i in range(0,len(contour_ids),contour_id_bunch_size):
    print('%i/%i'%(i,len(contour_ids)))
    ambiguous_contours_df=get_nodes_data(cursor,contour_ids[i:min(i+contour_id_bunch_size,len(contour_ids))])
    ambiguous_contours_df=ambiguous_contours_df.set_index('id')
    ambiguous_contours_df=ambiguous_contours_df.to_crs(osm_crs)
    ambiguous_contours_df['geometry']=ambiguous_contours_df['geometry'].apply(lambda ls:Polygon(ls))

    
    for j in range(i,min(i+contour_id_bunch_size,len(contour_ids))):
        id_contour=contour_ids[j]
        id_CC=random.choice(ambiguous_contours[id_contour])
        osm_node=random.choice(list(CC[id_CC]))
        datum=G_osm.nodes()[osm_node]
        pt=Point(datum['x'],datum['y'])
        if pt.within(ambiguous_contours_df.loc[id_contour]['geometry']):
            for id_CC in ambiguous_contours[id_contour]:
                CC_to_contours[id_CC]=id_contour
        else:
            predecessors=list(G.predecessors(id_contour))
            corrected_id_contour=id_contour if len(predecessors)==0 else predecessors[0]
            for id_CC in ambiguous_contours[id_contour]:
                CC_to_contours[id_CC]=corrected_id_contour        


t2=time.time()
print('ambiguities removed, took %f'%(t2-t1))

0/834
10/834
20/834
30/834
40/834
50/834
60/834
70/834
80/834
90/834
100/834
110/834
120/834
130/834
140/834
150/834
160/834
170/834
180/834
190/834
200/834
210/834
220/834
230/834
240/834
250/834
260/834
270/834
280/834
290/834
300/834
310/834
320/834
330/834
340/834
350/834
360/834
370/834
380/834
390/834
400/834
410/834
420/834
430/834
440/834
450/834
460/834
470/834
480/834
490/834
500/834
510/834
520/834
530/834
540/834
550/834
560/834
570/834
580/834
590/834
600/834
610/834
620/834
630/834
640/834
650/834
660/834
670/834
680/834
690/834
700/834
710/834
720/834
730/834
740/834
750/834
760/834
770/834
780/834
790/834
800/834
810/834
820/834
830/834
ambiguities removed, took 91.162791


In [21]:
G_osm.nodes(data=True)

NodeDataView({14468319: {'y': 4599879.701345091, 'x': 525532.7104872006, 'street_count': 3, 'lon': 9.3061536, 'lat': 41.5501734}, 15905397: {'y': 4600660.017360439, 'x': 526252.0983481632, 'street_count': 3, 'lon': 9.3148135, 'lat': 41.5571786}, 1434053206: {'y': 4599922.540843072, 'x': 525487.7483661725, 'street_count': 3, 'lon': 9.3056163, 'lat': 41.5505607}, 1434053161: {'y': 4599545.02578099, 'x': 525469.7844203033, 'street_count': 3, 'lon': 9.3053849, 'lat': 41.5471609}, 14468342: {'y': 4599321.181098565, 'x': 523840.15791965916, 'street_count': 3, 'lon': 9.2858369, 'lat': 41.5451949}, 510159636: {'y': 4599328.652947121, 'x': 523917.2417450709, 'street_count': 3, 'lon': 9.2867614, 'lat': 41.5452599}, 50488069: {'y': 4599339.866517927, 'x': 523763.012699509, 'street_count': 3, 'highway': 'mini_roundabout', 'lon': 9.2849127, 'lat': 41.5453655}, 2450736310: {'y': 4599295.527510675, 'x': 523861.32784904743, 'street_count': 1, 'lon': 9.2860897, 'lat': 41.5449632}, 14468344: {'y': 45991

In [22]:
t1=time.time()
osm_nodes_data={}
for k,cc in enumerate(CC):
    for osm_node in cc:
        osm_nodes_data[osm_node]={'contour_line_id':CC_to_contours[k],'geometry':Point(G_osm.nodes()[osm_node]['lon'],G_osm.nodes()[osm_node]['lat'])}
t2=time.time()
print(t2-t1)

0.3979356288909912


In [30]:
ADD_OSM_DATA=True
if ADD_OSM_DATA:
    
    t1=time.time()
    string_list=',\n'.join(['(%i,%i,ST_GeomFromText(\'%s\'))'%(osm_id,d['contour_line_id'],d['geometry'].wkt) for osm_id,d in osm_nodes_data.items()])
    cmd="INSERT INTO %s (`osm_id`,`contour_line_id`,`geometry`) VALUES %s;"%(osm_nodes_table_name,string_list)
    cursor.execute(cmd)

    
    intersection=intersection.to_crs('epsg:4326')
    string_list=',\n'.join(['(%i,%i,%i,%i,%f,%f,ST_GeomFromText(\'%s\'))'%(row['id'],row['edge'][0],row['edge'][1],row['edge'][2],row['edge_coordinate'],row['elevation'],row['geometry'].wkt)  for _,row in intersection.iterrows()])
    cmd="INSERT INTO %s (`contour_line_id`,`osm_begin`,`osm_end`,`osm_key`,`edge_coordinate`,`elevation`,`geometry`) VALUES %s;"%(osm_intersections_table_name,string_list)
    cursor.execute(cmd)
    t2=time.time()
    print(t2-t1)

141.03420782089233
